In [ ]:
#import libraries, dependencies

#Using a CSV file with all zip codes in Denver from:
#https://www.downloadexcelfiles.com/us_en/download-list-us-zip-codes-colorado-state#gsc.tab=0
#selected only zip codes in Denver City using excel

#TODO: import CSV and create a data frame

#TODO: select parameters for request to geoapify, which will find all healthcare facilities in each zip code

#TODO: Use a for loop to make an api request for each zip code, and place the healthcare facility results into a DF

#TODO: Draw a map which locates each healthcare facility (dot size population of the zip code? color by type of facility?)
 
#TODO: plot #of healthcare facilities per zip code against population per zip code (double bars with zip as x axis?)

#TODO: decide on at least four more visualizations to create

#TODO: written analysis

#TODO: prepare slides for presentation

#TODO: practice presentation(or at least be clear on respective roles)